# ANÁLISE DE FUNDOS DE INVESTIMENTO IMOBILIÁRIO (FII)

## INTRODUÇÃO


### Objetivo

Criar uma ferramenta que auxilie na tomada de decisão na hora de comprar um fundo imobiliário.

Essa ferramenta irá retornar uma tabela com os fundos que atendem determinados requisitos baseados em indicadores de FII.

A partir dessa tabela, será criado uma ferramenta de visualização dos dados de cada fundo.

### Dados necessários

Para esse objetivo, será necessário extrair dados com diversos indicadores de fundos imobiliários como Dividend Yield, Preço da cota sobre o Valor Patrimonial por cota, Valor de Mercado do fundo etc.

### Onde encontrar os dados

Os dados sobre indicadores foram retirados do site 'https://www.fundamentus.com.br/fii_resultado.php' através de web scraping.

## PSEUDO CÓDIGO

1. Conectar ao site com url = 'https://www.fundamentus.com.br/fii_resultado.php' realizando um webscraping para extrair os dados da tabela.
2. Exportar a tabela para um banco de dados ou para planilha Comma Separeted Values (CSV).
3. Filtrar os dados da seguinte forma:
    - filtrar por Dividend Yield;
    - filtrar por Preço/ValorPatrimonial;
    - filtrar por valor de mercado;
    - filtrar por taxa de vacância;
    - filtrar por liquidez.
4. Classificar os dados por DY(méd. anual);
5. Usar o tableau como ferramenta de visualização.

- Próximos passos:
    - filtrar tabela para conter apenas fundos que não precisam de investidor qualificado;
    - incluir um campo para que o usuário indique o valor que deseja investir e que retorne a combinação dos fundos que otimiza esse valor a ser investido;
    - incluir dados de localização dos imóveis para criar um mapa de com essas localidades.

## EXTRAINDO TABELA

### Instalando extensões

In [1]:
!pip3 install bs4
!pip3 install selenium
!pip3 install webdriver-manager

### Importando bibliotecas

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import matplotlib as plt
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

### WebScraping dos dados

Código para BeautifulSoup
def export_data(url):

    get_html = requests.get(url)

    html_content = get_html.content

    soup = BeautifulSoup(html_content, 'lxml')

    return soup

url = 'https://www.fundamentus.com.br/fii_resultado.php'

soup = export_data(url)

print(soup.prettify())

### Abrindo a página

In [3]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())
url = 'https://www.fundamentus.com.br/fii_resultado.php'
driver.get(url)
main_page = driver.current_window_handle

[WDM] - Downloading: 100%|██████████| 6.68M/6.68M [00:00<00:00, 18.4MB/s]
C:\Users\mathe\AppData\Local\Temp/ipykernel_12600/2665923747.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install())


### Extraindo tabela

In [4]:
rows = driver.find_elements(
    By.TAG_NAME, 'tr'
)
rows[-1].text

'ZIFI11 Híbrido 1.359,00 -1,38% 0,60% 1,13 33.621.700 537 0 0,00 0,00 0,00% 0,00%'

In [5]:
row = []
for tr in range(1,len(rows)):
    lista_td = driver.find_elements(By.XPATH, f'/html/body/div[1]/div[2]/table/tbody/tr[{tr}]//td')
    table_row = []
    for td in lista_td:
        table_row.append(td.text)
    row.append(table_row)

In [6]:
row[0]

['ABCP11',
 'Shoppings',
 '75,37',
 '9,00%',
 '8,69%',
 '0,82',
 '919.803.000',
 '103.432',
 '1',
 '12.898,50',
 '1.328,65',
 '10,30%',
 '14,43%',
 '']

th = []
for heading in driver.find_elements(By.TAG_NAME, 'th'):
    th.append(heading.text)

In [7]:
th = [
    'papel',
    'segmento',
    'cotação',
    'ffo',
    'dy',
    'p/vp',
    'valor_mercado',
    'liquidez',
    'num_imoveis',
    'preço_m2',
    'aluguel_m2',
    'cap_rate',
    'tx_vacancia',
    ''
]

In [8]:
len(th)

14

In [9]:
fii = pd.DataFrame(data=row, columns=th)

In [10]:
fii.head()

,papel,segmento,cotação,ffo,dy,p/vp,valor_mercado,liquidez,num_imoveis,preço_m2,aluguel_m2,cap_rate,tx_vacancia,
0,ABCP11,Shoppings,"75,37","9,00%","8,69%","0,82",919.803.000,103.432,1,"12.898,50","1.328,65","10,30%","14,43%",
1,AEFI11,Outros,"174,90","7,88%","0,00%","1,24",353.582.000,0,7,"4.058,74","331,87","8,18%","0,00%",
2,AFCR11,Híbrido,"103,15","11,42%","2,38%","1,00",186.437.000,0,0,"0,00","0,00","0,00%","0,00%",
3,AFHI11,Títulos e Val. Mob.,"100,83","7,66%","14,67%","1,06",249.719.000,734.288,0,"0,00","0,00","0,00%","0,00%",
4,AFOF11,Títulos e Val. Mob.,"91,94","10,22%","12,16%","0,94",63.548.200,138.000,0,"0,00","0,00","0,00%","0,00%",


In [11]:
fii.drop('', axis=1, inplace=True)

In [12]:
lista_replace = ['cotação', 'p/vp', 'valor_mercado', 'liquidez', 'num_imoveis', 'preço_m2', 'aluguel_m2']
for col in lista_replace:
    fii[col] = fii[col].map(lambda x: float(x.replace('.','').replace(',','.')))
lista_replace_p = ['ffo', 'dy', 'cap_rate', 'tx_vacancia']
for col in lista_replace_p:
    fii[col] = fii[col].map(lambda x: float(x.replace('%','').replace('.','').replace(',','.'))/100)

In [13]:
fii.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   papel          367 non-null    object 
 1   segmento       367 non-null    object 
 2   cotação        367 non-null    float64
 3   ffo            367 non-null    float64
 4   dy             367 non-null    float64
 5   p/vp           367 non-null    float64
 6   valor_mercado  367 non-null    float64
 7   liquidez       367 non-null    float64
 8   num_imoveis    367 non-null    float64
 9   preço_m2       367 non-null    float64
 10  aluguel_m2     367 non-null    float64
 11  cap_rate       367 non-null    float64
 12  tx_vacancia    367 non-null    float64
dtypes: float64(11), object(2)
memory usage: 37.4+ KB


## EXPORTANDO TABELA PARA BANCO DE DADOS

In [14]:
import os
from dotenv import load_dotenv
import sqlalchemy as sa

In [15]:
#load_dotenv('credentials/mysql.env')
#url_banco = "localhost"
#nome_db = "FII"
#conn_str = f"mysql+pymysql://{os.getenv('MYSQL_USER')}:{os.getenv('MYSQL_PASS')}@{url_banco}/{nome_db}"
#engine = sa.create_engine(conn_str)

In [16]:
#fii.dropna().to_sql('fii_completa', engine, index = False, if_exists = 'replace')
fii.dropna().to_csv('data/fii_completa.csv', decimal='.', date_format='DD/MM/YYYY', index=False, header = fii.columns)

## FILTRANDO DADOS

### Por Dividend Yield

In [17]:
print(fii['dy'].mean(), fii['dy'].max(), fii['dy'].min())

0.13017166212534056 10.425999999999998 0.0


In [18]:
fii.sort_values(by='dy', ascending=False)

,papel,segmento,cotação,ffo,dy,p/vp,valor_mercado,liquidez,num_imoveis,preço_m2,aluguel_m2,cap_rate,tx_vacancia
320,TOUR11,Residencial,8.80,0.0000,10.4260,0.00,0.0,0.0,3.0,0.00,0.00,0.0000,0.0000
50,BVAR11,Outros,100.01,0.8503,2.8182,0.10,62560300.0,0.0,64.0,473.72,439.38,0.9275,0.0621
189,LATR11B,Residencial,15.95,-0.1206,0.7994,1.45,1979460.0,8.0,0.0,0.00,0.00,0.0000,0.0000
266,RBRM11,Títulos e Val. Mob.,160000.00,0.5827,0.7229,3.40,86240000.0,0.0,0.0,0.00,0.00,0.0000,0.0000
158,IDFI11,Híbrido,69.84,0.0045,0.6803,0.96,143725000.0,17182.0,111.0,0.00,0.00,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,HBTT11,Títulos e Val. Mob.,1055.00,0.0653,0.0000,1.19,190126000.0,0.0,0.0,0.00,0.00,0.0000,0.0000
243,PRSV11,Lajes Corporativas,113.50,0.0644,0.0000,0.47,22132500.0,5229.0,1.0,1979.06,356.21,0.1800,0.4090
122,GWIR11,Shoppings,175.99,0.0280,0.0000,0.74,67231200.0,0.0,0.0,0.00,0.00,0.0421,0.0000
248,QIFF11,Títulos e Val. Mob.,97.49,0.0763,0.0000,1.15,14623500.0,0.0,0.0,0.00,0.00,0.0000,0.0000


Criando tabela com dados estatísticos para basear os filtros

In [19]:
stats = fii.describe().transpose()
stats['IQR'] = stats['75%'] - stats['25%']
#stats.dropna().to_sql('fii_stats', engine, index = False, if_exists = 'replace')
stats.dropna().to_csv('data/stats.csv', decimal='.', date_format='DD/MM/YYYY', header = stats.columns, index=False)
stats

,count,mean,std,min,25%,50%,75%,max,IQR
cotação,367.0,9.319347e+02,9.342367e+03,0.880,7.194000e+01,9.350000e+01,1.069300e+02,1.600000e+05,3.499000e+01
ffo,367.0,6.588311e-02,4.342889e-01,-7.585,4.360000e-02,7.630000e-02,1.049500e-01,2.133200e+00,6.135000e-02
dy,367.0,1.301717e-01,5.654471e-01,0.000,4.825000e-02,8.620000e-02,1.184500e-01,1.042600e+01,7.020000e-02
p/vp,367.0,2.723815e+00,2.602709e+01,0.000,8.200000e-01,9.400000e-01,1.020000e+00,4.863400e+02,2.000000e-01
valor_mercado,367.0,5.578099e+08,1.678909e+09,0.000,6.960810e+07,1.662980e+08,4.155780e+08,2.552000e+10,3.459699e+08
liquidez,367.0,6.895220e+05,1.408643e+06,0.000,5.596500e+03,1.142500e+05,6.019235e+05,1.118660e+07,5.963270e+05
num_imoveis,367.0,6.035422e+00,2.615321e+01,0.000,0.000000e+00,1.000000e+00,4.000000e+00,4.260000e+02,4.000000e+00
preço_m2,367.0,5.231413e+04,7.432386e+05,0.000,0.000000e+00,4.340000e+00,4.087260e+03,1.380730e+07,4.087260e+03
aluguel_m2,367.0,5.392734e+03,7.316425e+04,0.000,0.000000e+00,0.000000e+00,3.490500e+02,1.270991e+06,3.490500e+02
cap_rate,367.0,4.646866e-02,8.253139e-02,0.000,0.000000e+00,0.000000e+00,8.235000e-02,9.275000e-01,8.235000e-02


In [20]:
stats.loc['dy', 'IQR']

0.0702

In [21]:
stats.columns

Index(['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'IQR'], dtype='object')

Nesta etapa vamos filtrar os dados para pegar Fundos com Dividend Yield maior que 10% e menores que o terceiro quartil mais 1,5xIQR (Interquartile range)

In [22]:
dy_min = stats.loc['dy', '75%']
dy_max = stats.loc['dy', '75%'] + stats.loc['dy', 'IQR'] * 1.5
mask_dy = (fii['dy'] < dy_max) & (fii['dy'] > dy_min)

In [23]:
dy_max

0.22375

In [24]:
fii_1 = fii[mask_dy]
fii_1.dropna().to_csv('data/f1.csv', decimal='.', date_format='DD/MM/YYYY', index=False, header = fii_1.columns)
fii_1.head()

,papel,segmento,cotação,ffo,dy,p/vp,valor_mercado,liquidez,num_imoveis,preço_m2,aluguel_m2,cap_rate,tx_vacancia
3,AFHI11,Títulos e Val. Mob.,100.83,0.0766,0.1467,1.06,249719000.0,734288.0,0.0,0.0,0.00,0.0000,0.0
4,AFOF11,Títulos e Val. Mob.,91.94,0.1022,0.1216,0.94,63548200.0,138000.0,0.0,0.0,0.00,0.0000,0.0
12,ARCT11,Híbrido,103.41,0.1037,0.1679,1.03,383462000.0,1900930.0,9.0,393.6,21.81,0.0554,0.0
13,ARRI11,Títulos e Val. Mob.,8.96,1.4656,0.1749,0.10,10557200.0,322767.0,0.0,0.0,0.00,0.0000,0.0
15,BARI11,Títulos e Val. Mob.,97.31,0.1452,0.1506,0.95,450953000.0,1060110.0,1.0,0.0,0.00,0.0000,0.0


### Por 'Preço / Valor Patrimonial' (P/VP)

Esse índice nos mostra se um fundo está sendo comercializado por um bom valor. Se o P/VP for menor que 1, significa que a cota está sendo negociada por um preço abaixo do valor patrimonial por cota do fundo, o que significa que a cota está "barata".

Dessa forma, vamos procurar por fundos que possuem p/vp abaixo do terceiro quartil com o intuito de comprar fundos que tenham chances de valoriza, minimizando a perda caso seja necessário vendê-lo futuramente.
Também não podemos escolher uma cota com p/vp muito baixo, pois isso pode ser um indicador de que o fundo está sendo mal administrado. Portanto, vamos pegar apenas cotas com p/vp acima do segundo quartil.

In [25]:
pvp_min = stats.loc['p/vp', '50%']
pvp_max = stats.loc['p/vp', '75%']
mask_pvp = (fii_1['p/vp'] < pvp_max) & (fii_1['p/vp'] > pvp_min)

In [26]:
fii_2 = fii_1[mask_pvp]
fii_2.dropna().to_csv('data/f2.csv', decimal='.', date_format='DD/MM/YYYY', index=False, header = fii_2.columns)
len(fii_2)

26

### Por Valor de Mercado

Com este filtro pretendemos excluir da tabela os fundos pequenos, pois estamos visando fundos mais estáveis e bem administrados para corrermos menos riscos.

In [27]:
vm_min = stats.loc['valor_mercado', '50%']
mask_vm = (fii_2['valor_mercado'] > vm_min)

In [28]:
fii_3 = fii_2[mask_vm]
fii_3.dropna().to_csv('data/f3.csv', decimal='.', date_format='DD/MM/YYYY', index=False, header = fii_3.columns)
len(fii_3)

20

In [29]:
fii_3

,papel,segmento,cotação,ffo,dy,p/vp,valor_mercado,liquidez,num_imoveis,preço_m2,aluguel_m2,cap_rate,tx_vacancia
15,BARI11,Títulos e Val. Mob.,97.31,0.1452,0.1506,0.95,4.509530e+08,1060110.0,1.0,0.00,0.0,0.0,0.0
24,BCRI11,Títulos e Val. Mob.,104.06,0.1214,0.1461,0.98,6.511940e+08,989325.0,0.0,0.00,0.0,0.0,0.0
46,BTCR11,Títulos e Val. Mob.,94.75,0.1060,0.1313,1.00,4.557570e+08,951493.0,0.0,0.00,0.0,0.0,0.0
51,CACR11,Híbrido,101.62,0.1159,0.1591,0.99,1.836710e+08,267737.0,0.0,0.00,0.0,0.0,0.0
71,DEVA11,Títulos e Val. Mob.,96.78,0.0785,0.1617,0.98,1.345800e+09,3768780.0,0.0,0.00,0.0,0.0,0.0
93,FEXC11,Títulos e Val. Mob.,87.63,0.1224,0.1333,0.95,5.348500e+08,977764.0,0.0,0.00,0.0,0.0,0.0
124,HABT11,Títulos e Val. Mob.,94.90,0.0736,0.1640,0.95,7.712320e+08,1934660.0,0.0,0.00,0.0,0.0,0.0
183,KNHY11,Títulos e Val. Mob.,96.60,0.0690,0.1586,1.00,1.786220e+09,2612020.0,0.0,0.00,0.0,0.0,0.0
184,KNIP11,Títulos e Val. Mob.,91.84,0.0621,0.1707,0.97,7.372880e+09,11186600.0,0.0,0.00,0.0,0.0,0.0
187,KNSC11,Títulos e Val. Mob.,85.60,0.0558,0.1744,0.97,1.136410e+09,4300430.0,0.0,0.00,0.0,0.0,0.0


In [30]:
len(fii_3)

20

### Por Taxa de Vacância

A taxa de vacância indica a porcentagem de quantos metros quadrados do fundo não estão pagando aluguel, ou seja, se um fundo possui 1000 m² e 100m² não estão alugados, esse fundo terá uma taxa de vacância de 10%.

Dessa forma, se for o caso, queremos excluir fundos com altas taxas de vacância.

In [31]:
tx_max = 0.05
mask_tx = (fii_3['valor_mercado'] > tx_max)

In [32]:
fii_4 = fii_3[mask_tx]
fii_4.dropna().to_csv('data/f4.csv', decimal='.', date_format='DD/MM/YYYY', index=False, header = fii_4.columns)
len(fii_4)

20

### Por liquidez

A liquidez de um fundo representa a quantidade de cotas que estão sendo negociadas em um dia. O que representa um indicador de quão rápido conseguimos comprar e vender uma cota desse fundo.

Buscamos fundos com alta liquidez, pois não queremos perder tempo tentando comprar ou vender uma cota.

In [33]:
lq_min = stats.loc['liquidez', '75%']
mask_lq = fii_4['liquidez'] > lq_min

In [34]:
fii_5 = fii_4[mask_lq]
fii_5.dropna().to_csv('data/f5.csv', decimal='.', date_format='DD/MM/YYYY', index=False, header = fii_5.columns)

In [35]:
fii_5

,papel,segmento,cotação,ffo,dy,p/vp,valor_mercado,liquidez,num_imoveis,preço_m2,aluguel_m2,cap_rate,tx_vacancia
15,BARI11,Títulos e Val. Mob.,97.31,0.1452,0.1506,0.95,4.509530e+08,1060110.0,1.0,0.0,0.0,0.0,0.0
24,BCRI11,Títulos e Val. Mob.,104.06,0.1214,0.1461,0.98,6.511940e+08,989325.0,0.0,0.0,0.0,0.0,0.0
46,BTCR11,Títulos e Val. Mob.,94.75,0.1060,0.1313,1.00,4.557570e+08,951493.0,0.0,0.0,0.0,0.0,0.0
71,DEVA11,Títulos e Val. Mob.,96.78,0.0785,0.1617,0.98,1.345800e+09,3768780.0,0.0,0.0,0.0,0.0,0.0
93,FEXC11,Títulos e Val. Mob.,87.63,0.1224,0.1333,0.95,5.348500e+08,977764.0,0.0,0.0,0.0,0.0,0.0
124,HABT11,Títulos e Val. Mob.,94.90,0.0736,0.1640,0.95,7.712320e+08,1934660.0,0.0,0.0,0.0,0.0,0.0
183,KNHY11,Títulos e Val. Mob.,96.60,0.0690,0.1586,1.00,1.786220e+09,2612020.0,0.0,0.0,0.0,0.0,0.0
184,KNIP11,Títulos e Val. Mob.,91.84,0.0621,0.1707,0.97,7.372880e+09,11186600.0,0.0,0.0,0.0,0.0,0.0
187,KNSC11,Títulos e Val. Mob.,85.60,0.0558,0.1744,0.97,1.136410e+09,4300430.0,0.0,0.0,0.0,0.0,0.0
228,OUJP11,Híbrido,96.11,0.1409,0.1455,0.95,3.125870e+08,703896.0,0.0,0.0,0.0,0.0,0.0


## Classificando os dados

Nesta etapa vamos classificar os dados por Dividend Yield para observar as melhores oportunidades.

In [36]:
fii_6 = fii_5.sort_values(by='dy', ascending=False)
fii_6

,papel,segmento,cotação,ffo,dy,p/vp,valor_mercado,liquidez,num_imoveis,preço_m2,aluguel_m2,cap_rate,tx_vacancia
187,KNSC11,Títulos e Val. Mob.,85.60,0.0558,0.1744,0.97,1.136410e+09,4300430.0,0.0,0.0,0.0,0.0,0.0
335,VGIP11,Títulos e Val. Mob.,91.57,0.0895,0.1735,1.00,1.079360e+09,3288450.0,0.0,0.0,0.0,0.0,0.0
184,KNIP11,Títulos e Val. Mob.,91.84,0.0621,0.1707,0.97,7.372880e+09,11186600.0,0.0,0.0,0.0,0.0,0.0
124,HABT11,Títulos e Val. Mob.,94.90,0.0736,0.1640,0.95,7.712320e+08,1934660.0,0.0,0.0,0.0,0.0,0.0
71,DEVA11,Títulos e Val. Mob.,96.78,0.0785,0.1617,0.98,1.345800e+09,3768780.0,0.0,0.0,0.0,0.0,0.0
328,VCJR11,Títulos e Val. Mob.,92.77,0.0977,0.1605,0.99,1.365940e+09,1700110.0,0.0,0.0,0.0,0.0,0.0
183,KNHY11,Títulos e Val. Mob.,96.60,0.0690,0.1586,1.00,1.786220e+09,2612020.0,0.0,0.0,0.0,0.0,0.0
280,RECR11,Títulos e Val. Mob.,91.80,0.1421,0.1568,0.96,2.427340e+09,5085390.0,0.0,0.0,0.0,0.0,0.0
239,PORD11,Títulos e Val. Mob.,96.50,0.1358,0.1513,0.98,3.597880e+08,890600.0,0.0,0.0,0.0,0.0,0.0
344,VRTA11,Títulos e Val. Mob.,93.15,0.1304,0.1508,0.99,1.225160e+09,2420020.0,0.0,0.0,0.0,0.0,0.0


In [37]:
len(fii_6)

17

In [38]:
fii_6.to_csv('data/fii.csv', decimal='.', date_format='DD/MM/YYYY', index=True, header = fii_6.columns)

## Webscrapping para dados históricos

Para analisarmos as melhores opções, vamos precisar fazer um webscrapping dos dados históricos das 5 primerias ações da tabela

In [39]:
driver.get(url)

In [40]:
input_tag = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/form/fieldset/input[1]')
input_tag.send_keys(fii_6.iloc[0,0])

In [41]:
send_button = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/form/fieldset/input[2]')
send_button.click()

In [42]:
hist_button = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[3]/ul[2]/li[3]/ul/li[4]/a')
hist_button = hist_button.get_attribute('href')
driver.get(hist_button)

In [43]:
for i in range(0,5):
    input_tag = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/form/fieldset/input[1]')
    input_tag.send_keys(fii_6.iloc[i,0])
    send_button = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/form/fieldset/input[2]')
    send_button.click()
    hist_button = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[3]/ul[2]/li[3]/ul/li[4]/a')
    hist_button = hist_button.get_attribute('href')
    driver.get(hist_button)
    rows = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div/table/tbody')
    rows = rows.find_elements(By.TAG_NAME, 'tr')
    row = []
    for tr in rows:
        tr = tr.text + f' {fii_6.iloc[i,0]}'
        row.append(tr.split(' '))
    colunas = ['ultima_data_com', 'tipo', 'data_pgto', 'valor', 'tag']
    tag_hist = pd.DataFrame(data=row, columns=colunas)
    tag_hist['valor'] = tag_hist['valor'].map(lambda x: float(x.replace('.','').replace(',','.')))
    #tag_hist.dropna().to_sql(f'tag{i}', engine, index = False, if_exists = 'replace')
    tag_hist.dropna().to_csv(f'data/tag{i}.csv', decimal='.', date_format='DD/MM/YYYY')

In [44]:
tag0 = pd.read_csv('data/tag0.csv').drop('Unnamed: 0', axis=1)
tag0

,ultima_data_com,tipo,data_pgto,valor,tag
0,31/08/2022,Rendimento,14/09/2022,0.70,KNSC11
1,29/07/2022,Rendimento,11/08/2022,0.91,KNSC11
2,30/06/2022,Rendimento,13/07/2022,1.09,KNSC11
3,31/05/2022,Rendimento,13/06/2022,1.35,KNSC11
4,29/04/2022,Rendimento,12/05/2022,1.14,KNSC11
5,31/03/2022,Rendimento,13/04/2022,1.12,KNSC11
6,25/02/2022,Rendimento,14/03/2022,1.00,KNSC11
7,31/01/2022,Rendimento,11/02/2022,1.25,KNSC11
8,31/01/2022,Rendimento,11/01/2022,1.25,KNSC11
9,30/12/2021,Rendimento,13/01/2022,1.45,KNSC11


In [45]:
tag0.iloc[0:12,:].groupby(by='tag').mean()

,valor
tag,
KNSC11,1.163333


In [46]:
tag0.iloc[0:12,:]

,ultima_data_com,tipo,data_pgto,valor,tag
0,31/08/2022,Rendimento,14/09/2022,0.70,KNSC11
1,29/07/2022,Rendimento,11/08/2022,0.91,KNSC11
2,30/06/2022,Rendimento,13/07/2022,1.09,KNSC11
3,31/05/2022,Rendimento,13/06/2022,1.35,KNSC11
4,29/04/2022,Rendimento,12/05/2022,1.14,KNSC11
5,31/03/2022,Rendimento,13/04/2022,1.12,KNSC11
6,25/02/2022,Rendimento,14/03/2022,1.00,KNSC11
7,31/01/2022,Rendimento,11/02/2022,1.25,KNSC11
8,31/01/2022,Rendimento,11/01/2022,1.25,KNSC11
9,30/12/2021,Rendimento,13/01/2022,1.45,KNSC11


In [47]:
tag0.iloc[0:12,:].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
valor,12.0,1.163333,0.217437,0.7,1.0675,1.195,1.3125,1.45
